Explanations

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feedparser
from bs4 import BeautifulSoup
import urllib
import urlparse

from geopy.geocoders import GoogleV3, Nominatim

import folium

%matplotlib inline

# Preliminary

This function wraps the geocoder

In [4]:
def geocodewrapper(df):
    
    for index, row in df.iterrows():
        
        if row['geocodestatus'] == 0:
        
            try:
                ans = geolocator.geocode(row['address'])
            except IndexError:
                print 'No address data for id ' + row['id']
                ans = None

            if ans == None:
                print 'Could not encode: ' + row['address']
            else:
                df.set_value(index, 'lat', ans.latitude)
                df.set_value(index, 'long', ans.longitude)
                df.set_value(index, 'geocodestatus', 1)
    
    return df

# Combine data on donation drives

## Haema

In [5]:
haema = pd.read_excel('static/data_in/haema_fixedlocations.xlsx', encoding='latin-1')

First round using OSM Nominatim encoder

In [6]:
geolocator = Nominatim()
haema_encoded = geocodewrapper(haema)

Could not encode: Oststr. 51, 99867 Gotha


Second round using Google encoder

In [7]:
geolocator = GoogleV3()
haema_encoded = geocodewrapper(haema_encoded)

Export

In [8]:
haema_encoded.to_csv('static/data_out/haema_fixedlocations_encoded.csv', encoding='latin-1')

## German Red Cross Donation Centers

In [9]:
drkfixed = pd.read_excel('static/data_in/drk_fixedlocations.xlsx', encoding='latin-1')

In [10]:
geolocator = Nominatim()
drkfixed_encoded = geocodewrapper(drkfixed)

Could not encode: Sperlichstr. 15, 48151 Münster
Could not encode: Klarissengasse 9, 48151 Münster


Second round using Google encoder

In [11]:
geolocator = GoogleV3()
drkfixed_encoded = geocodewrapper(drkfixed_encoded)

Export to CSV with coordinates

In [12]:
drkfixed_encoded.to_csv('static/data_out/drk_fixedlocations_encoded.csv', encoding='latin-1')

## Hospitals

In [22]:
hospitals = pd.read_excel('static/data_in/hospitals_fixedlocations.xlsx', encoding='latin-1')

Only one round using OSM Nominatim encoder

In [23]:
geolocator = Nominatim()
hospitals_encoded = geocodewrapper(hospitals)

Could not encode: Carl-Neuberg-Str. 1, 30625 Hannover
Could not encode: Ratzeburger Allee 160, 23538 Lübeck


Second round using Google encoder

In [24]:
geolocator = GoogleV3()
hospitals_encoded = geocodewrapper(hospitals_encoded)

Export to CSV with coordinates

In [25]:
hospitals_encoded.to_csv('static/data_out/hospitals_fixedlocations_encoded.csv', encoding='latin-1')

## Append together

In [26]:
fixedlocations = pd.concat([hospitals_encoded,drkfixed_encoded,haema_encoded], ignore_index=True)
fixedlocations.to_csv('static/data_out/fixedlocations_encoded.csv', encoding='latin-1')

### Map fixed locations

In [27]:
START_COORD = (50.6807, 10.217)
map = folium.Map(location=START_COORD, zoom_start=5,tiles='OpenStreetMap')

# Add markers from CSV file
for drive in fixedlocations.iterrows():
    popuptext = 'Name: ' + drive[1]['name']

    if drive[1]['bsd'] == 'hospital':
         folium.Marker([drive[1]['lat'], drive[1]['long']],
              popup=popuptext,
              icon=folium.Icon(icon='certificate',color='blue')
             ).add_to(map)
    elif drive[1]['bsd'] == 'haema':
        folium.Marker([drive[1]['lat'], drive[1]['long']],
              popup=popuptext,
              icon=folium.Icon(icon='asterisk',color='purple')
             ).add_to(map)
    else:
         folium.Marker([drive[1]['lat'], drive[1]['long']],
              popup=popuptext,
              icon=folium.Icon(icon='tint',color='red')
             ).add_to(map)

In [28]:
map